In [1]:
# This script following step2_cluster_analysis.ipynb plots personalized networks
import sys
sys.path.append('../network')
import network
import svg_draw as sd
import pandas as pd
import os
from reportlab.graphics import renderPDF
from svglib.svglib import svg2rlg

def draw_network(fr, keystone_df, layout_df, max_edge_width = 6, max_r = 16, min_width = 1, cutoff=0.9):
    origin = (515.0, 515.0)
    common_taxon = list(set(fr.index).intersection(set(layout_df.index)))
    fr = fr.loc[common_taxon, common_taxon]
    layer0_df = keystone_df[keystone_df['layer'] == 0]
    keystone_cluster = list(keystone_df[(keystone_df['layer'] == 1) & (keystone_df['is_keystone']==True)]['leaves'])[0].split(',')
    circles = ''
    keystone = ''
    pr_scores = layer0_df['PR_score']
    pr_max = max(pr_scores)
    pr_range = pr_max
    xy_df = pd.DataFrame()
    for idx in common_taxon:
        theta = layout_df.loc[idx, 'theta']
        r = layout_df.loc[idx, 'r']
        color = layout_df.loc[idx, 'color']
        x, y = network.convert(theta, r, origin)
        xy_df.loc[idx, 'x'] = x
        xy_df.loc[idx, 'y'] = y
        pr = layer0_df.loc[idx, 'PR_score']
        if idx in keystone_cluster:
            swidth = 4
        else:
            swidth = 1
        if layer0_df.loc[idx, 'is_keystone'] == True:
            keystone = idx
            circle = sd.draw_circle(x, y, swidth=swidth, r=max_r, fill="red", id=idx, style=True)
        else:
            r = max_r*pr/pr_range
            circle = sd.draw_circle(x, y, swidth=swidth, r=r, fill=color, id=idx, style=True)
        circles +=  circle
    
    paths = ""
    ntaxon = fr.shape[0]
    taxon = list(fr.index)
    for i in range(ntaxon):
        taxa1 = taxon[i]
        x1 = xy_df.loc[taxa1, 'x']
        y1 = xy_df.loc[taxa1, 'y']
        for j in range(i+1, ntaxon):
            taxa2 = taxon[j]
            fr_value = fr.loc[taxa1, taxa2]
            if fr_value > 0.9:
                x2 = xy_df.loc[taxa2, 'x']
                y2 = xy_df.loc[taxa2, 'y']
                id = "{}_{}".format(taxa1, taxa2)
                x3, y3 = network.compute_convex((x1, y1), (x2, y2))
                edge_width = max_edge_width*(fr_value-cutoff)/(1-cutoff)
                if taxa1 == keystone or taxa2 == keystone:
                    path = sd.draw_curve(x1, y1, x3, y3, x2, y2, stroke='red', swidth=edge_width, id=id, style=True)
                else:
                    path = sd.draw_curve(x1, y1, x3, y3, x2, y2, swidth=edge_width, id=id, style=True)
                paths += path
    title = '<text x="15" y="40" id="sp">{}</text>'.format(keystone)
    return title + paths + circles




In [2]:
legend = '<circle cx="150" cy="1060" r="12" stroke="black" stroke-width="1" fill="red" /> \n \
<text x="170" y="1065" class="legend">keystone</text>\n \
<circle cx="350" cy="1060" r="12" stroke="black" stroke-width="1" fill="#85D696" />\n \
<text x="370" y="1065" class="legend">p__Firmicutes</text>\n \
<circle cx="550" cy="1060" r="12" stroke="black" stroke-width="1" fill="#7EC9CE" />\n \
<text x="570" y="1065" class="legend">p__Bacteroidetes</text>\n \
<circle cx="750" cy="1060" r="12" stroke="black" stroke-width="1" fill="#FFC68A" />\n \
<text x="770" y="1065" class="legend">p__Proteobacteria</text>\n \
<circle cx="150" cy="1090" r="12" stroke="black" stroke-width="1" fill="#999FDB" />\n \
<text x="170" y="1095" class="legend">p__Actinobacteria</text>\n \
<circle cx="350" cy="1090" r="12" stroke="black" stroke-width="1" fill="#C0E481" />\n \
<text x="370" y="1095" class="legend">p__Spirochaetes</text>\n \
<circle cx="550" cy="1090" r="12" stroke="black" stroke-width="1" fill="#CC99DB" />\n \
<text x="570" y="1095" class="legend">p__Euryarchaeota</text>\n \
<circle cx="750" cy="1090" r="12" stroke="black" stroke-width="1" fill="#AAAAAA" />\n \
<text x="770" y="1095" class="legend">Other</text>'

def draw(content, legend):
    width = 1050
    height = 1110
    style = "\
        text {\n \
            font-family: Arial;\n\
        }\n\
        .legend {\n\
            font-size: 20px;\n\
        }\n\
        #sp {\n\
            font-size: 22px;\n\
            font-weight: bold;\n\
        }\n"
    
    return sd.canvas(width, height, content+legend, style)



In [3]:

layout_df = pd.read_csv('../draw_network/init/sector_sp_layout.tsv', sep='\t', header=0, index_col=0)
outer_dir = '../result/pheno_result/'

for d in os.listdir(outer_dir):
    in_dir = os.path.join(outer_dir, d)
    if not os.path.isdir(in_dir):
        continue
    for dir in os.listdir(in_dir):
        sp_dir = os.path.join(in_dir, dir, 'sp')
        for enterotype_dir in os.listdir(sp_dir):
            keystone_df = pd.read_csv(os.path.join(sp_dir, enterotype_dir, 'keystone_node.tsv'), sep='\t', header=0, index_col=0)
            fr_df = pd.read_csv(os.path.join(sp_dir, enterotype_dir, 'layer_0', 'fr.tsv'), sep='\t', header=0, index_col=0)
            s_sector= draw(draw_network(fr_df, keystone_df, layout_df), legend)
            svg_dir = os.path.join(sp_dir, enterotype_dir, 'network.svg')
            with open(svg_dir, 'w') as fp:
                fp.write(s_sector)

            drawing = svg2rlg(svg_dir)
            pdf_path = os.path.join(sp_dir, enterotype_dir, 'network.pdf')
            renderPDF.drawToFile(drawing, pdf_path)
            